In [ ]:
import cv2
import os
import numpy as np

def extract_frames_from_videos(video_folder, output_folder, mode="interval", frame_skip=15, diff_thresh=10):
    """
    Extract frames from all videos in a folder.

    Args:
        video_folder (str): Path to the folder containing videos.
        output_folder (str): Path where extracted frames will be saved.
        mode (str): "interval" or "motion"
        frame_skip (int): Save 1 frame every 'frame_skip' frames (for interval mode).
        diff_thresh (float): Threshold for detecting scene change (for motion mode).
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(video_folder):
        if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            video_path = os.path.join(video_folder, filename)
            video_name = os.path.splitext(filename)[0]
            save_path = os.path.join(output_folder, video_name)

            if not os.path.exists(save_path):
                os.makedirs(save_path)

            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            saved_count = 0
            prev_hist = None

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                if mode == "interval":
                    if frame_count % frame_skip == 0:
                        frame_filename = os.path.join(save_path, f"{video_name}_frame{frame_count}.jpg")
                        cv2.imwrite(frame_filename, frame)
                        saved_count += 1

                elif mode == "motion":
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
                    hist = cv2.normalize(hist, hist).flatten()

                    if prev_hist is None:
                        prev_hist = hist
                        frame_filename = os.path.join(save_path, f"{video_name}_frame{frame_count}.jpg")
                        cv2.imwrite(frame_filename, frame)
                        saved_count += 1
                    else:
                        diff = cv2.compareHist(prev_hist, hist, cv2.HISTCMP_BHATTACHARYYA) * 100
                        if diff > diff_thresh:
                            frame_filename = os.path.join(save_path, f"{video_name}_frame{frame_count}.jpg")
                            cv2.imwrite(frame_filename, frame)
                            saved_count += 1
                            prev_hist = hist  # update reference histogram

                frame_count += 1

            cap.release()
            print(f"Extracted {saved_count} frames from {filename} into {save_path}")

# Example usage:
video_folder = "videos"          # Folder containing your .mp4, .avi etc.
output_folder = "imgs"  # Where to save extracted frames

# Choose mode:
# "interval" → every Nth frame
# "motion"   → based on scene change
extract_frames_from_videos(video_folder, output_folder, mode="motion", frame_skip=15, diff_thresh=15)


Extracted 74 frames from croc_1.mp4 into imgs\croc_1
Extracted 1 frames from croc_plat_1.mp4 into imgs\croc_plat_1
Extracted 43 frames from croc_plat_2.mp4 into imgs\croc_plat_2
Extracted 19 frames from kangaroo_1.mp4 into imgs\kangaroo_1
Extracted 78 frames from kangaroo_2.mp4 into imgs\kangaroo_2


KeyboardInterrupt: 